# PDM Data Summary



## TODO:

- distribution of # of slides per patient and sample (when relevant) from HoBBIT down the PDM tables
- compare the distribution of various columns from HoBBIT down the PDM tables 
 - overall, stratified by cancer type
 - number of slides, part_type, part_description stain_name, stain_group, scanner_id, brand, model, image_id 	magnification
- Look at strange values for the above fields, and see if there are values in the HoBBIT table that aren't in the subsequent PDM tables or vice versa
- using the above, put together questions for Luke
 - The duplicate slide IDs, invalid MRNs
 - 

In [ ]:

import os
import pandas as pd
import sys
import getpass
import matplotlib.pyplot as plt
import numpy as np
sys.path.append("../")

from connector import DremioDataframeConnector
pd.set_option('display.max_columns', None)

FONT_BASE = {
    "family": "sans-serif",
    "sans-serif": "helvetica",
    "weight": "normal",
    "size": 18,
}

plt.rc("font", **FONT_BASE)
plt.rc("axes", unicode_minus=False)
from matplotlib import rcParams
plt.rcParams.update({'figure.autolayout': True})


def mrn_zero_pad(df: pd.DataFrame, col_mrn: str) -> pd.DataFrame:
    """zero pad MRN columns"""
    df[col_mrn] = df[col_mrn].astype(str).str.zfill(8)
    return df

def plot_hist(df, col_sample, col_patient):
    """plots # of samples/slides per patient"""
    fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8, 8)) #, figsize=(32,32))

    df_sample = df.groupby(by=[col_patient])[col_sample].nunique()\
        .reset_index(name='count')\
        .sort_values(['count'], ascending=False).reset_index()
    
    bins = np.arange(1,df_sample['count'].max()+1)
    counts, edges, bars = ax1.hist(df_sample['count'], bins=bins)
    ax1.set_xticks(bins)
    plt.bar_label(bars)
    plt.title(f"Number of {col_sample} per {col_patient}")
    plt.show()

In [ ]:
# Setup Dremio connector
# Credentials (also could be read via .env)

DREMIO_USER = input("Username: ")
DREMIO_PASSWORD = getpass.getpass(prompt="Password or PAT: ", stream=None)

dremio_session = DremioDataframeConnector(
   scheme="grpc+tcp",
   hostname="tlvidreamcord1",
   flightport=32010,
   dremio_user=DREMIO_USER,
   dremio_password=DREMIO_PASSWORD,
   connection_args={},
)



## Loading data sources

Following code blocks are for loading in data sources (hobbit & IMPACT/CDM) and doing preliminary filtering and plotting

In [ ]:
query = 'select * from "hobbit-poc"."case_breakdown"'
df_hobbit = dremio_session.get_table(query)
display(df_hobbit)

In [ ]:
# Processing df_hobbit like in HoBBIT-case-breakdown-summary
df_hobbit = df_hobbit.drop_duplicates()
ids = df_hobbit['image_id']
df_stain_dups = df_hobbit[ids.isin(ids[ids.duplicated()])].sort_values("image_id")
df_hobbit = df_hobbit[~df_hobbit['mrn'].isin(df_stain_dups['mrn'])]
# also removing incorrect MRNs
invalid_mrns = ['bsbsbs', 'GP102021LC', 'DMP12345']
df_hobbit = df_hobbit[~(df_hobbit['mrn'].isin(invalid_mrns))]
display(df_hobbit)

In [ ]:
print(f"Number of patients: {len(df_hobbit['mrn'].unique())}")
print(f"Number of slides: {len(df_hobbit['image_id'].unique())}")
print(f"Number of rows: {len(df_hobbit)}")
print(f"Number of rows (removing duplicates): {len(df_hobbit.drop_duplicates())}")

In [ ]:


query = 'select * from impact."daily_data_clinical_sample.oncokb.txt"'
df_sample = dremio_session.get_table(query)

display(df_sample)

query = 'select * from impact."daily_data_clinical_patient.solid_heme.txt"'
df_patient = dremio_session.get_table(query).drop(0)

display(df_patient)

df_impact = df_sample.merge(df_patient, on="PATIENT_ID", how='inner')
display(df_impact)

In [ ]:
# removing CFDNA samples

df_impact = df_impact[df_impact['SAMPLE_CLASS']=='Tumor']

print(f"Number of patients: {len(df_impact['PATIENT_ID'].unique())}")
print(f"Number of samples: {len(df_impact['SAMPLE_ID'].unique())}")
print(f"Number of rows: {len(df_impact)}")
print(f"Number of rows (removing duplicates): {len(df_impact.drop_duplicates())}")


In [ ]:
# distribution of # of samples per patient
plot_hist(df_impact, col_sample='SAMPLE_ID',col_patient='PATIENT_ID')

### Notes on cdm pathology_impact_sample_summary_dop table

- two samples that were flagged are already known, where the sample IDs cooresponded to different MRNs
- table is larger than # of cases w/ IMPACT on cbioportal because of reasons outlined: 
 - patients couldn've been consented to multiple pannels (i.e. Archer)
 - Patients only had partial consent to 12-245

In [ ]:
query = 'select * from phi_data_lake."cdm-data".pathology."table_pathology_impact_sample_summary_dop_anno.tsv"'
df_cdm = dremio_session.get_table(query)
df_cdm = mrn_zero_pad(df_cdm, "MRN")
# invalid SAMPLE IDS flagged by Chris - coorespond to multple MRNs
invalid_dmp_ids = ['P-0000000-N-VR1', 'P-0032211-T02-IM6']
df_cdm = df_cdm[~df_cdm['SAMPLE_ID'].isin(invalid_dmp_ids)]
display(df_cdm)

print(f"Number of patients: {len(df_cdm['MRN'].unique())}")
print(f"Number of samples: {len(df_cdm['SAMPLE_ID'].unique())}")
print(f"Number of rows: {len(df_cdm)}")
print(f"Number of rows (removing duplicates): {len(df_cdm.drop_duplicates())}")


In [ ]:
df_cdm['SOURCE_ACCESSION_NUMBER_0'] = df_cdm['SOURCE_ACCESSION_NUMBER_0'].replace('', np.nan)
df_cdm['SOURCE_SPEC_NUM_0'] = df_cdm['SOURCE_SPEC_NUM_0'].replace('', np.nan)

df_cdm = df_cdm.dropna(subset=['SOURCE_ACCESSION_NUMBER_0'])
df_cdm = df_cdm.dropna(subset=['SOURCE_SPEC_NUM_0'])

display(df_cdm)
print(f"Number of patients: {len(df_cdm['MRN'].unique())}")
print(f"Number of samples: {len(df_cdm['SAMPLE_ID'].unique())}")
print(f"Number of rows: {len(df_cdm)}")
print(f"Number of rows (removing duplicates): {len(df_cdm.drop_duplicates())}")



In [ ]:
plot_hist(df_cdm, col_sample='SAMPLE_ID',col_patient='MRN')


In [ ]:
df_impact = df_impact.merge(df_cdm, on='SAMPLE_ID', how='inner')
display(df_impact)
print(f"Number of patients (MRN): {len(df_impact['MRN'].unique())}")
print(f"Number of patients (Patient ID): {len(df_impact['PATIENT_ID'].unique())}")
print(f"Number of samples: {len(df_impact['SAMPLE_ID'].unique())}")
print(f"Number of rows: {len(df_impact)}")
print(f"Number of rows (removing duplicates): {len(df_impact.drop_duplicates())}")

In [ ]:
# relative drop off in number of IMPCAT samples per patient when inner joined on IMPACT table
plot_hist(df_impact, col_sample='SAMPLE_ID',col_patient='MRN')


## PDM Data Merges

Subsequent codeblocks are for merging data between IMPACT/CDM and HoBBIT and analyzing the results

In [ ]:
display(df_hobbit)
df_merge_patient = df_impact.merge(df_hobbit, left_on='MRN', right_on='mrn', how='inner') #, how='left')
display(df_merge_patient)
print(f"Number of patients (MRN): {len(df_merge_patient['MRN'].unique())}")
print(f"Number of patients (Patient ID): {len(df_merge_patient['PATIENT_ID'].unique())}")
print(f"Number of samples: {len(df_merge_patient['SAMPLE_ID'].unique())}")
print(f"Number of slides: {len(df_merge_patient['image_id'].unique())}")

print(f"Number of rows: {len(df_merge_patient)}")
print(f"Number of rows (removing duplicates): {len(df_merge_patient.drop_duplicates())}")


# print(len(set(df_merge_patient['MRN'].to_list())))
# print(len(set(df_merge_patient['SAMPLE_ID'].to_list())))
# print(len(set(df_merge_patient['image_id'].to_list())))


In [ ]:
df_merge_accession = df_impact.merge(df_hobbit, left_on=['MRN', 'SOURCE_ACCESSION_NUMBER_0'], right_on=['mrn','specnum_formatted'], how='inner') #, how='left')
print(f"Number of patients (MRN): {len(df_merge_accession['MRN'].unique())}")
print(f"Number of patients (Patient ID): {len(df_merge_accession['PATIENT_ID'].unique())}")
print(f"Number of samples: {len(df_merge_accession['SAMPLE_ID'].unique())}")
print(f"Number of slides: {len(df_merge_accession['image_id'].unique())}")

print(f"Number of rows: {len(df_merge_accession)}")
print(f"Number of rows (removing duplicates): {len(df_merge_accession.drop_duplicates())}")



In [ ]:
df_hobbit['part_inst'] = df_hobbit['part_inst'].astype(str)
df_merge_part = df_impact.merge(df_hobbit, left_on=['MRN', 'SOURCE_ACCESSION_NUMBER_0', 'SOURCE_SPEC_NUM_0'], right_on=['mrn','specnum_formatted', 'part_inst'], how='inner') #, how='left')
# df_hobb
display(df_merge_part)
print(f"Number of patients (MRN): {len(df_merge_part['MRN'].unique())}")
print(f"Number of patients (Patient ID): {len(df_merge_part['PATIENT_ID'].unique())}")
print(f"Number of samples: {len(df_merge_part['SAMPLE_ID'].unique())}")
print(f"Number of slides: {len(df_merge_part['image_id'].unique())}")

print(f"Number of rows: {len(df_merge_part)}")
print(f"Number of rows (removing duplicates): {len(df_merge_part.drop_duplicates())}")



In [ ]:
df_merge_part